<a href="https://www.kaggle.com/code/carloscll/votingclassifier-steel-plate-defect?scriptVersionId=167326502" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# basics
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# sckit
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

# sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


# Reprod
SEED = 101

# path
URL = '/kaggle/input/playground-series-s4e3/train.csv'

# targert
TARGET_FEATURES = [
    
                'Pastry', 
                'Z_Scratch', 
                'K_Scatch', 
                'Stains',
                'Dirtiness', 
                'Bumps', 
                'Other_Faults'
    ]


# load
train = pd.read_csv(URL).set_index('id')

In [2]:
# seed
np.random.seed(SEED)


# features and labels
X = train.drop(TARGET_FEATURES, axis=1)
y = train[TARGET_FEATURES]


# train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)


# models 
models = {

        'nn': MLPClassifier(max_iter = 1000),
        'lgr': LogisticRegression(max_iter = 1000),
        'exc':  ExtraTreesClassifier(),
        'dtc': DecisionTreeClassifier(),
        'rfc': RandomForestClassifier(),
        'nbc': GaussianNB(),
        'knn': KNeighborsClassifier(),
        'gbc': GradientBoostingClassifier(),
        #'lgbm': LGBMClassifier(),
        'xgb': XGBClassifier(),
        'cat': CatBoostClassifier(logging_level='Silent'),
        'ada': AdaBoostClassifier()
}

def train_eval(models, X_train, X_val, y_train, y_val):
    '''
    Function to evaluate the
    models

    models: desired models
    X_train: training feature
    X_val: validation feature
    y_train: training label
    y_val: validation label

    '''
    
    numeric_features = X.select_dtypes(exclude=['object']).columns
    numeric_transformer = Pipeline(
        steps=[("scaler", RobustScaler())]) 
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),])
    
    models_score = {}
    for name, model in models.items():
        
        clf = Pipeline(steps=[("preprocessor", preprocessor),("classifier", model)]) # models w/ fe
        treinar = MultiOutputClassifier(clf).fit(X_train, y_train) # models wo/ fe
        models_score[name] = treinar.score(X_val, y_val)
    
    return models_score


nn_ense = MLPClassifier(max_iter = 1000)
knn_ens = KNeighborsClassifier()
xgb_ens = XGBClassifier()
cat_ens = CatBoostClassifier(logging_level='Silent')
ada_ens = AdaBoostClassifier()


voting_clf = VotingClassifier(
    estimators=[
        ('knn', knn_ens),
        #('nn', nn_ense),
        #('ada', ada_ens), 
        ('cat', cat_ens),
        ('xgb', xgb_ens)], voting='hard')


numeric_features = X.select_dtypes(exclude=['object']).columns
numeric_transformer = Pipeline(
    steps=[("scaler", RobustScaler())]) 

    
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),])

clf = Pipeline(steps=[
     ("preprocessor", preprocessor),
                       ("classifier", voting_clf)])

clf_vcl = MultiOutputClassifier(clf).fit(X_train, y_train)    

**Modelos**

- 'nn': 0.4032258064516129,
- 'knn': 0.42468782518210196,
- 'xgb': 0.4192247658688866,
- 'cat': 0.4171436004162331,
- 'ada': 0.3913891779396462}

# Display

In [3]:
clf_vcl

MultiOutputClassifier(estimator=Pipeline(steps=[('preprocessor',
                                                 ColumnTransformer(transformers=[('num',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   RobustScaler())]),
                                                                                  Index(['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
       'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A30...
                                                                                             grow_policy=None,
                                                                                             importance_type=None,
                                                                                             interaction_constraints=None,
                                                                                             learning_rate=None,
                                                                                             max_bin=None,
                                                                                             max_cat_threshold=None,
                                                                                             max_cat_to_onehot=None,
                                                                                             max_delta_step=None,
                                                                                             max_depth=None,
                                                                                             max_leaves=None,
                                                                                             min_child_weight=None,
                                                                                             missing=nan,
                                                                                             monotone_constraints=None,
                                                                                             multi_strategy=None,
                                                                                             n_estimators=None,
                                                                                             n_jobs=None,
                                                                                             num_parallel_tree=None,
                                                                                             random_state=None, ...))]))]))

In [4]:
clf_vcl.score(X_val, y_val)

0.41649323621227885

In [5]:
# # Make predictions
# y_pred_proba = np.array(clf_vcl.predict_proba(X_val))[:, :, 1]

# # Calculate ROC curve and AUC
# fpr, tpr, _ = metrics.roc_curve(y_val.values.ravel(), y_pred_proba.ravel())
# auc = metrics.roc_auc_score(y_val.values.ravel(), y_pred_proba.ravel())

# #create ROC curve
# plt.plot(fpr,tpr,label="AUC="+str(auc))
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.legend(loc=4)
# plt.show()

# Sub

In [6]:
# np.random.seed(101)

# test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv').set_index('id')
# sub = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')
# sub[TARGET_FEATURES] = clf_vcl.predict(test)
# sub.to_csv('20__ensemble__fe.csv', index=False)